# Acordo de chaves para comunicação confidencial e autenticada

## Connection

In [1]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Primitivas criptográficas

O uso do esquema Encrypt-then-MAC permitiu garantir a autenticidade, integridade e confidencialidade das mensagens trocadas entre o `Emitter` e o `Receiver`.
Como tal, a cifra simétrica escolhida foi o AES (Advanced Encryption Standard) em modo CFB (Cipher Feedback Mode).
O MAC (Message Authentication Code) escolhido foi o HMAC (Hash-MAC) com SHA-256 que permitiu garantir a integridade
e autenticidade da mensagem trocada.

In [7]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import algorithms, modes

# DH Parameters
P = 99494096650139337106186933977618513974146274831566768179581759037259788798151499814653951492724365471316253651463342255785311748602922458795201382445323499931625451272600173180136123245441204133515800495917242011863558721723303661523372572477211620144038809673692512025566673746993593384600667047373692203583
G = 44157404837960328768872680677686802650999163226766694797650810379076416463147265401084491113667624054557335394761604876882446924929840681990106974314935015501571333024773172440352475358750668213444607353872754650805031912866692119819377041901642732455911509867728218394542745330014071040326856846990119719675

def verifyHMAC(key, tag, ctxt):
    """ Computes the HMAC of a given cipher text and checkes whether it matches with
        the given tag i.e. validates the authenticity and integrity of the message
    """
    try:
        h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
        h.update(ctxt)
        h.verify(tag)                        #exception on MAC failure
        return True
    except:
        return False

def extractMessage(key, iv, tweak_h, ctxt):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    decryptor = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend()).decryptor()
    ptxt = (decryptor.update(ctxt ^ tweak_h) + decryptor.finalize()) ^ tweak_h
    return ptxt

## Emitter

In [3]:
def Emitter(conn):
    pwd = getpass('Emitter password: ')
    try:
        #TODO: key exchange DH
        iv = os.urandom(12)
        #TODO: AES in CFB
        #TODO: hash tweak . SHA256
        cipher = Cipher(algorithms.AES(key),modes.GCM(iv),backend).encryptor()

        header = 'msgheader'.encode("utf-8")
        content = 'Here is a sample message for testing'
        #TODO: XOR w/ msg & hash tweak 
        cipher.authenticate_additional_data(header)
        content = cipher.update(content.encode("utf-8")) + cipher.finalize()
        #TODO: XOR w/ cryptogram & hash tweak
        payload = {'header': header, 'content': content, 'iv':iv, 'tag': cipher.tag}

        conn.send(payload)
        conn.close()
    except:
        print("Emitter: Error")

## Receiver

O Receiver, após a troca de chaves, e cada mensagem recebida, efetua uma verificação da autenticidade e integridade da mesma com recurso à função `verifyHMAC` que recebe como parâmetro a chave usada no cálculo do MAC, o MAC esperado e o *input* sobre o qual este deve ser calculado.

Caso a mensagem recebida não tenha sido modificada, o `Receiver` procede ao processo de decifragem. Dado tratar-se de
uma mensagem cifrada com **TAES** é necessário manipular o criptograma de modo a obter a mensagem original, sendo útil ter em conta a seguinte expressão para o processo de decifragem:

$$ ctxt = E_k(ptxt \oplus h(tweak)) \oplus h(tweak)$$

onde:
- $ctxt$ : criptograma recebido
- $ptxt$ : mensagem original
- $tweak$: tweak/nonce

como tal, e tendo ainda em conta que $ D_k = E^{-1}_k $ , a obtenção do texto limpo original($ptxt$) pode ser conseguida da seguinte maneira:

$$ ptxt = D_k(ctxt \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak)) \oplus h(tweak) \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak))) \oplus h(tweak) \rightarrow ptxt \oplus h(tweak) \oplus h(tweak) \rightarrow ptxt $$

Este processo é implementado pela função `extractMessage` que recebe como parâmetros a chave acordada (`key`), o vetor de inicialização (`iv`), o *hash* do *tweak* (`tweak_h`) e o criptograma (`ctxt`), devolvendo o resultado de aplicar a expressão anteriormente apresentada.


In [6]:
def Receiver(conn):
    pwd = getpass('Reciever password: ')
    try:
        #TODO: key exchange
        payload = conn.recv()
        iv = payload['iv']
        m_hmac = payload['m_hmac']
        header = payload['header']
        ctxt = payload['content']
        if verifyHMAC(h_key, m_hmac, header+ctxt):
            msg = extractMessage(c_key, ctxt, tweak_h)
            print("Received: " + msg.decode("utf-8"))
        conn.close()
    except:
        print("Receiver: Erro na decifragem")

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)